In [ ]:
import csv
import json
from xml.etree import ElementTree as et
from pony.orm import Database, db_session, get, select, Required, PrimaryKey, flush

In [ ]:
#create a function to read any xml file
def read_xml(filename):
    #using the ElementTree library, parse the xml data into a variable
    xml_tree = et.parse(filename) 
    #get the root element of the XML tree
    root = xml_tree.getroot() 
    #extracts the attributes of each element in the root of the XML tree
    data = [values.attrib for values in root] 
    return data
xml_data = read_xml('user_data.xml')

In [ ]:
#create a function to read any csv file
def csv_reader(filename):
    with open(filename, mode ='r') as csv_file:
        # read the csv data into the set variable utilizing the csv library
        users_csv_file = csv.DictReader(csv_file, delimiter= ',')  
        data = [elements for elements in users_csv_file]
    return data
csv_data = csv_reader('user_data.csv')

In [ ]:
#create a function to read any json file
def load_json(filename):
    with open(filename) as json_file:
        #use json load atrribute to load json data
        data = json.load(json_file)   
    return data
json_data  = load_json('user_data.json')

In [ ]:

csv_dict = []
for rows in csv_data:  
    #create a dictionary for each row. map the keys to the corresponding values 
    csv_dic = {      
        'firstname': rows['First Name'],
        'lastname': rows['Second Name'],
        'age': rows['Age (Years)'],
        'sex': rows['Sex'],
        'vehiclemake': rows['Vehicle Make'],
        'vehiclemodel': rows['Vehicle Model'],
        'vehicleyear': rows['Vehicle Year'],
        'vehicletype': rows['Vehicle Type']       
        }
    csv_dict.append(csv_dic)   #append the dictionary to the csv_dict list as the loop runs through

In [ ]:
xml_dict = []
for rows in xml_data:   #iterate over the rows of data 
    
    #fix empty dependants observed from try error @db insert
        rows['dependants'] = int(rows['dependants']) if rows['dependants'] else 0
        #create a dictionary for each row. map the keys to the corresponding values
        xml_dic = {     
            'firstname': rows['firstName'],
            'lastname': rows['lastName'],
            'age': rows['age'],
            'dependants': rows['dependants'],
            'marital_status': rows['marital_status'],
            'salary': rows['salary'],
            'company': rows['company'],
            'pension': rows['pension'],
            'address_postcode': rows['address_postcode'],
            'sex': rows['sex'],
            'commute_distance': rows['commute_distance'],
            'retired': rows['retired']                    
            }
        xml_dict.append(xml_dic)    #append the dictionary to the xml_dict list as the loop runs through

In [ ]:
json_dict = []
for rows in json_data:
        try:
            if not type(rows['debt']) is float:
                     raise TypeError('only floats allowed')
            json_dict.append(rows)
        except KeyError: 
#             print('some keys are absent')
            pass
        except Exception as errs: 
#             print(errs)
            pass

        #get values for debt, if None, create a new dic with values, for the str conver them to dic and create time_period_years. 
        debt = rows.get('debt',{'amount':0,'time_period_years':0})
        if type(debt) != dict:
            debt = {'amount':float(debt),'time_period_years':0}
        rows['debt'] = debt
      
        #create a dictionary for each row. map the keys to the corresponding values        
        json_dic = {
                'firstname': rows['firstName'],
                'lastname': rows['lastName'],
                'age': rows['age'],
                'address_main': rows['address_main'],
                'credit_card_end_date': rows['credit_card_end_date'],
                'debt': rows['debt']['amount'],
                'time_period_years': rows['debt']['time_period_years'],
                'iban': rows['iban'],
                'address_postcode': rows['address_postcode'],
                'credit_card_number': rows['credit_card_number'],
                'credit_card_start_date': rows['credit_card_start_date'],
                'credit_card_security_code': rows['credit_card_security_code'],
                'address_city': rows['address_city']                    
                }

        json_dict.append(json_dic)

In [ ]:
combined_dict = csv_dict.copy() 

# function to check if the current item in the xml_dict or json_dict already exists in the combined_dict
def check_data(mainlist, nxt_list):
    for i in range(0, len(mainlist)):
        if mainlist[i].get('firstname') == nxt_list.get('firstname') and mainlist[i].get('lastname') == nxt_list.get('lastname') and str(mainlist[i].get('age')) == str(nxt_list.get('age')):
            return i
    return -1


In [ ]:
# iterate over json_dict and update or append items to combined_dict
for items in json_dict:
    #return the index position
    index = check_data(combined_dict, items)
    #for the index position, update the fields from the json_dict dictionary
    if index != -1:
        combined_dict[index].update(items)
        
    else:
        combined_dict.append(items)

In [ ]:

# iterate over xml_dict and update or append items to combined_dict
for item in xml_dict:
    # check if the current item already exists in the combined_dict and return the index
    index = check_data(combined_dict, item)
    # if the item already exists, update the matched item
    if index != -1:
        combined_dict[index].update(item)
    # if the item does not exist, append the item to the combined_dict
    else:
        combined_dict.append(item)
#sort dictionary
combined_dict.sort(key=lambda x: (x['firstname'], x['lastname']))

In [ ]:
# setup database connection using PonyORM
db = Database()
db.bind(provider='mysql', host='europa.ashley.work', user='student_xxxxx', passwd='xxxxxxx', db='student_xxxxx')


In [ ]:
# define class as db entity, table structure, set decorators and schema
class Customers(db.Entity):
    id = PrimaryKey(int, auto=True)
    firstname = Required(str)
    lastname = Required(str)
    age = Required(int)

    sex = Required(str)
    marital_status = Required(str)
    salary = Required(float)
    company = Required(str)
    commute_distance = Required(float)
    pension = Required(float)
    retired = Required(str)
    dependants = Required(int)
    vehiclemake = Required(str)
    vehiclemodel = Required(str)
    vehicleyear = Required(int)
    vehicletype = Required(str)
    iban = Required(str)
    debt = Required(float)
    time_period_years = Required(int)
    credit_card_number = Required(str)
    credit_card_start_date = Required(str)
    credit_card_end_date = Required(str)
    credit_card_security_code = Required(int)
    address_main = Required(str)
    address_city = Required(str)
    address_postcode = Required(str)

db.generate_mapping(create_tables=True)


In [ ]:
#delete existing receords seen in db
with db_session:
    Customers.select().delete()

In [ ]:
#inserting the field and records into the DB
with db_session:
    for item in combined_dict:
        try:
            db.insert(Customers, firstname=item['firstname'], lastname=item['lastname'], age=item['age'], sex=item['sex'], marital_status=item['marital_status'], salary=item['salary'], company=item['company'], commute_distance=item['commute_distance'], pension=item['pension'], retired=item['retired'], dependants=item['dependants'], vehiclemake=item['vehiclemake'], vehiclemodel=item['vehiclemodel'], vehicleyear=item['vehicleyear'], vehicletype=item['vehicletype'], iban=item['iban'], credit_card_number=item['credit_card_number'], credit_card_security_code=item['credit_card_security_code'], credit_card_start_date=item['credit_card_start_date'], credit_card_end_date=item['credit_card_end_date'], address_main=item['address_main'], address_city=item['address_city'], address_postcode=item['address_postcode'], debt=item['debt'], time_period_years=item['time_period_years']) 
        except Exception as db_err: print(db_err)

Task1 "Shane Chambers e-mailed in overnight (Full details in Ticket #1839). During account creation something went wrong and their security code is wrong on their billing information. Bank is rejecting any payment until it's corrected. They're not sure what happened, but said to try "935". Can you please action this and try re-bill the client? Let me know if there's any further issues with it. Thanks"

In [ ]:
with db_session:
    customer = Customers.get(firstname ='Shane', lastname ='Chambers')   #select a customer with the specified 'firstname' and 'lastname'
    print(customer.firstname, customer.lastname, customer.credit_card_security_code)
    customer.set(credit_card_security_code = '935')
    print(customer.firstname, 'new security code is:', customer.credit_card_security_code)
    flush()
    print('Shane Chambers security code has been corrected accordingly')


Task2 "Congratulations on the promotion Lane! We wouldn't have survived through the pandemic without you and your team. As a token of our appreciation, we've given you a £2100 salary bump. This will take effect as of next month's payroll. At Lewis-Johnson we value the care and work you put in. See you on Monday Joshua, enjoy!"

In [ ]:
# The goal of this task is to increase the salary of Lane by £2100 due to promotion.
# get a Lane, or a Joshua or a Joshua lane that works with Lewis-Johnson

with db_session:
    customers = select(c for c in Customers if c.firstname == 'lane' or c.lastname.lower() =='lane' and c.company == 'Lewis-Johnson')
    for c in customers:
        print( f'name: {c.firstname} {c.lastname} | company: {c.company} | salary: {c.salary}')


In [ ]:
# from the result 
# name: Joshua Lane | company: Lewis-Johnson | salary: 97949.0
# name: Dorothy Lane | company: Wong, Rose and Johnston | salary: 84421.0

# Joshua Lane is the target since he is the "Lane" that works with Lewis-Johnson

with db_session:
    c = Customers.get(firstname= 'Joshua', lastname = 'Lane', company = 'Lewis-Johnson')
    print(c.firstname, c.lastname, ', works for', c.company)
    c.salary += 2100
    flush()
print(c.lastname, 'salary updated to:', c.salary)

Task3 "Happy Birthday Ms Suzanne Wright! You're 37 today. Our latest offers will be sure to get you into the party spirit!"

In [ ]:
with db_session:
    c = Customers.get(firstname='Suzanne', lastname='Wright')
    c.age = 37
    flush()
print('The age of ', c.firstname + " " + c.lastname, ' has been updated to ', c.age)

Task 4 "Hannah, the pension policy has changed slightly since the meeting last week. I know you've just finished putting through all the changes, but I need you to look at Mr Dunn's file. We'll need to  modify it by adding another 0.15% on top of the existing £22358. Drop me a message when you've done this please, it's quite urgent."

In [ ]:
with db_session:
    # find a customer with first or last name  Dunn and pension 22391.537
    #after updating the pension, the and pension condition has to be removed
    customer2 = select(c for c in Customers if c.firstname == 'Dunn' or c.lastname == 'Dunn' and c.pension == 22358.0) 
    for c in customer2:
        print(c.firstname, c.lastname, c.pension)
    flush()

In [ ]:
with db_session:
    
    c = Customers.get(firstname='Kyle', lastname ='Dunn')
    c.pension = c.pension * (1 + 0.15/100)
    print(c.firstname, c.lastname, ' pension modified by 0.15% to ', c.pension)
    flush()